In [139]:
from pynq import Overlay

ol = Overlay("./matrixmult.bit")

In [140]:
ol.ip_dict

{'transfer_kernel_0': {'fullpath': 'transfer_kernel_0',
  'type': 'xilinx.com:hls:transfer_kernel:1.0',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1073741824,
  'mem_id': 's_axi_CTRL',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CTRL_ADDR_WIDTH': '7',
   'C_S_AXI_CTRL_DATA_WIDTH': '32',
   'C_M_AXI_MATA_ID_WIDTH': '1',
   'C_M_AXI_MATA_ADDR_WIDTH': '64',
   'C_M_AXI_MATA_DATA_WIDTH': '32',
   'C_M_AXI_MATA_AWUSER_WIDTH': '1',
   'C_M_AXI_MATA_ARUSER_WIDTH': '1',
   'C_M_AXI_MATA_WUSER_WIDTH': '1',
   'C_M_AXI_MATA_RUSER_WIDTH': '1',
   'C_M_AXI_MATA_BUSER_WIDTH': '1',
   'C_M_AXI_MATA_USER_VALUE': '0x00000000',
   'C_M_AXI_MATA_PROT_VALUE': '"000"',
   'C_M_AXI_MATA_CACHE_VALUE': '"0011"',
   'C_M_AXI_MATB_ID_WIDTH': '1',
   'C_M_AXI_MATB_ADDR_WIDTH': '64',
   'C_M_AXI_MATB_DATA_WIDTH': '32',
   'C_M_AXI_MATB_AWUSER_WIDTH': '1',
   'C_M_AXI_MATB_ARUSER_WIDTH': '1',
   'C_M_AXI_MATB_WUSER_WIDTH': '1',
   'C_M_AXI_MAT

In [141]:
ip = ol.transfer_kernel_0

In [142]:
ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0, CHAN2_INT_EN=0, RESERVED_1=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0, CHAN2_INT_ST=0, RESERVED_1=0),
  matA_offset_1 = Register(matA_offset=0),
  matA_offset_2 = Register(matA_offset=0),
  matB_offset_1 = Register(matB_offset=0),
  matB_offset_2 = Register(matB_offset=0),
  matout_offset_1 = Register(matout_offset=0),
  matout_offset_2 = Register(matout_offset=0),
  mat1r = Register(mat1r=0),
  mat1cmat2r = Register(mat1cmat2r=0),
  mat2c = Register(mat2c=0)
}

In [143]:
from pynq import allocate
import numpy as np

np.set_printoptions(suppress=True)

mat1r = 100
mat1c_mat2r = 100
mat2c = 100

inbuff = allocate(shape=(mat1r*mat1c_mat2r,), dtype=np.float32)
in2buff = allocate(shape=(mat1c_mat2r*mat2c,), dtype=np.float32)
outbuff = allocate(shape=(mat1r*mat2c,), dtype=np.float32)

In [144]:
print("inbuff physical address {}".format(hex(inbuff.physical_address)))
print("in2buff physical address {}".format(hex(in2buff.physical_address)))
print("outbuff physical address {}".format(hex(outbuff.physical_address)))

inbuff physical address 0x169a0000
in2buff physical address 0x169b0000
outbuff physical address 0x169c0000


In [145]:
ip.register_map.matA_offset_1 = inbuff.physical_address
ip.register_map.matB_offset_1 = in2buff.physical_address
ip.register_map.matout_offset_1 = outbuff.physical_address
ip.register_map.mat1r = mat1r
ip.register_map.mat1cmat2r = mat1c_mat2r
ip.register_map.mat2c = mat2c

In [146]:
acc = 0.25

for i in range(10000):
    inbuff[i] = acc
    acc+=0.25

acc = 0

for i in range(100):
    for j in range(i, 10000, 100):
        in2buff[j] = (j*0.25)+0.25

with np.printoptions(threshold=np.inf):
    print(in2buff)
#in2buff

[   0.25    0.5     0.75    1.      1.25    1.5     1.75    2.      2.25
    2.5     2.75    3.      3.25    3.5     3.75    4.      4.25    4.5
    4.75    5.      5.25    5.5     5.75    6.      6.25    6.5     6.75
    7.      7.25    7.5     7.75    8.      8.25    8.5     8.75    9.
    9.25    9.5     9.75   10.     10.25   10.5    10.75   11.     11.25
   11.5    11.75   12.     12.25   12.5    12.75   13.     13.25   13.5
   13.75   14.     14.25   14.5    14.75   15.     15.25   15.5    15.75
   16.     16.25   16.5    16.75   17.     17.25   17.5    17.75   18.
   18.25   18.5    18.75   19.     19.25   19.5    19.75   20.     20.25
   20.5    20.75   21.     21.25   21.5    21.75   22.     22.25   22.5
   22.75   23.     23.25   23.5    23.75   24.     24.25   24.5    24.75
   25.     25.25   25.5    25.75   26.     26.25   26.5    26.75   27.
   27.25   27.5    27.75   28.     28.25   28.5    28.75   29.     29.25
   29.5    29.75   30.     30.25   30.5    30.75   31.     3

In [147]:
outbuff

PynqBuffer([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [148]:
ip.register_map.CTRL.AP_START=1

In [149]:
import time

while ip.register_map.CTRL.AP_DONE == 0:
    time.sleep(1)

print(ip.register_map.CTRL.AP_DONE)

0


In [150]:

#with np.printoptions(threshold=np.inf):
#    print(outbuff)
#outbuff

In [151]:
arrA = np.array(inbuff)
arr2dA = np.reshape(arrA, (mat1r,mat1c_mat2r))
np.savetxt('mm_matA.csv', arr2dA, delimiter=',', fmt='%f')

arrB = np.array(in2buff)
arr2dB = np.reshape(arrB, (mat1c_mat2r,mat2c))
np.savetxt('mm_matB.csv', arr2dB, delimiter=',', fmt='%f')

arr = np.array(outbuff)
arr2d = np.reshape(arr, (mat1r,mat2c))
np.savetxt('mm_result.csv', arr2d, delimiter=',', fmt='%f')

In [152]:
ip.register_map.CTRL.AUTO_RESTART=1

In [153]:
del inbuff, in2buff, outbuff